In [1]:
import pandas as pd
import copy
%load_ext autotime

In [2]:
game_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "2016_17_NBA_Scores")
team_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "Division_Info")

time: 164 ms


In [3]:
def create_game_dicts(game_data):
    games = []
    for index, row in game_data.iterrows():
        game = {}
        if row['Winner'] == 'Home':
            game[row['Home Team']] = 'Winner'
            game[row['Away Team']] = 'Loser'
        else:
            game[row['Home Team']] = 'Loser'
            game[row['Away Team']] = 'Winner'
        games.append(game)
    return games

time: 5.28 ms


In [4]:
def simulate_games(games, teams):
    for game in games:
        for team in game:
            teams[team]['Games_Played'] += 1
            if game[team] == 'Loser':
                teams[team]['Max_Wins'] -= 1

time: 2.19 ms


In [5]:
def initialize_team_standings(team_data):
    teams = {}
    for index, row in team_data.iterrows():
        teams[row['Team_Name']] = {'Division': row['Division_id'], 'Conference': row['Conference_id'], 'Games_Played': 0, 
                                  'Max_Wins': 82}
    return teams

time: 2.39 ms


In [6]:
def get_league_data(team_data):
    east_divisions = set(team_data[team_data['Conference_id'] == 'East']['Division_id'])
    west_divisions = set(team_data[team_data['Conference_id'] == 'West']['Division_id'])
    east_teams = set(team_data[team_data['Conference_id'] == 'East']['Team_Name'])
    west_teams = set(team_data[team_data['Conference_id'] == 'West']['Team_Name'])
    return {'East': {'Divisions': east_divisions, 'Teams': east_teams}, 
            'West': {'Divisions': west_divisions, 'Teams': west_teams}}

time: 3.49 ms


In [8]:
league_data = get_league_data(team_data)
team_standings = initialize_team_standings(team_data)
days = []
first_game_date = game_data.iloc[0,0]
first_game_index = 0
for index, row in game_data.iloc[0:].iterrows():
    if row['Date'] != first_game_date:
        days.append(DayOfGames(game_data.iloc[first_game_index:index], team_standings, league_data))
        first_game_index = index
        first_game_date = row['Date']
days.append(DayOfGames(game_data.iloc[first_game_index:index+1], team_standings, league_data))

time: 319 ms


In [7]:
class DayOfGames(object):
    def __init__(self, data, team_standings, league_data):
        self.data = data
        self.league_data = league_data
        self.date = self.data.iloc[0,0]
        self.games = self.create_game_dicts()
        self.team_standings_universal = team_standings
        self.add_game_results()
        self.team_standings = copy.deepcopy(self.team_standings_universal)
        
    def create_game_dicts(self):
        games = []
        for index, row in self.data.iterrows():
            game = {}
            if row['Winner'] == 'Home':
                game[row['Home Team']] = 'Winner'
                game[row['Away Team']] = 'Loser'
            else:
                game[row['Home Team']] = 'Loser'
                game[row['Away Team']] = 'Winner'
            games.append(game)
        return games    
        
    def add_game_results(self):
        for game in self.games:
            for team in game:
                self.team_standings_universal[team]['Games_Played'] += 1
                if game[team] == 'Loser':
                    self.team_standings_universal[team]['Max_Wins'] -= 1

time: 14 ms
